In [7]:
!pip install datasets
import transformers
import datasets
from datasets import load_dataset


In [8]:
loaded_data = load_dataset("dair-ai/emotion")
print(loaded_data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [11]:
##just
print(loaded_data["train"][0])
print(loaded_data["validation"][101])
print(loaded_data["test"][0])

{'text': 'i didnt feel humiliated', 'label': 0}
{'text': 'im feeling nostalgic cant beat the corys iframe allowfullscreen allowfullscreen frameborder height src http www', 'label': 2}
{'text': 'im feeling rather rotten so im not very ambitious right now', 'label': 0}


In [ ]:
###questions to do how to check the number of calsses
print(newtokendata["train"].features["text"])

In [12]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenizenewdataset(loaded_data):
  return tokenizer(loaded_data["text"],padding = True,truncation = True)
newtokendata = loaded_data.map(tokenizenewdataset)
print(newtokendata)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})


In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)


In [27]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from transformers import TrainingArguments,Trainer
trainingarguments = TrainingArguments(output_dir = "test-trainer",
                                      per_device_train_batch_size = 8,
                                      per_device_eval_batch_size = 8,
                                      eval_strategy = "epoch",
                                      num_train_epochs = 3,
                                      weight_decay = 0.01,
                                      logging_dir = "./logs",
                                      learning_rate = 1e-5,
                                      load_best_model_at_end = True,
                                      save_strategy = "epoch",
                                      save_total_limit = 3,
                                      metric_for_best_model = "eval_loss",
                                      report_to = "none")

In [35]:
trainer = Trainer(model = model,
                  args = trainingarguments,
                  data_collator = data_collator,
                  tokenizer = tokenizer,
                  train_dataset = newtokendata["train"],
                  eval_dataset =newtokendata["validation"])
trainer.train()

<ipython-input-35-e99a607a91bc>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model,args = trainingarguments,data_collator = data_collator,tokenizer = tokenizer,train_dataset = newtokendata["train"],eval_dataset =newtokendata["validation"])


Epoch,Training Loss,Validation Loss
1,0.165200,0.244651
2,0.168800,0.205155
3,0.111400,0.225918


TrainOutput(global_step=6000, training_loss=0.14628721300760905, metrics={'train_runtime': 356.6475, 'train_samples_per_second': 134.587, 'train_steps_per_second': 16.823, 'total_flos': 513070413095232.0, 'train_loss': 0.14628721300760905, 'epoch': 3.0})

In [36]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.20515505969524384, 'eval_runtime': 2.7896, 'eval_samples_per_second': 716.96, 'eval_steps_per_second': 89.62, 'epoch': 3.0}


In [37]:
predictions = trainer.predict(newtokendata["test"])
print(predictions)

PredictionOutput(predictions=array([[ 7.8017297, -1.6398602, -2.7376297, -1.4995509, -2.5907667,
        -3.1648386],
       [ 8.083832 , -2.0372388, -2.8947215, -1.5591823, -2.14341  ,
        -3.2915485],
       [ 7.902491 , -1.8470455, -2.6826885, -1.8610446, -2.206323 ,
        -3.0964057],
       ...,
       [-2.581884 ,  7.8931565, -1.713913 , -2.7112021, -3.3388321,
        -2.2927349],
       [-2.2740576,  7.646452 , -1.8312378, -2.8512611, -3.0056539,
        -2.3871844],
       [-2.4858642, -3.0070012, -2.8610668, -2.6060083,  4.425935 ,
         3.26094  ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 4]), metrics={'test_loss': 0.24941205978393555, 'test_runtime': 3.5431, 'test_samples_per_second': 564.474, 'test_steps_per_second': 70.559})


In [40]:
### now converting the logits to label classes we got 6 logits per 1 row because we are classifyingto 6 classes.pick the highest score .predictions return tuples
import numpy as np
preds = np.argmax(predictions.predictions,axis = 1)

(2000,)


In [43]:
label_names = loaded_data["train"].features["label"].names
print(label_names)
label_names1 = loaded_data["train"].features["label"]
print(type(label_names1))

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
<class 'datasets.features.features.ClassLabel'>


In [49]:
import sklearn
from sklearn.metrics import accuracy_score,classification_report,f1_score
accuracy  = accuracy_score(preds,predictions.label_ids)
print(accuracy)
label_names = loaded_data["train"].features["label"].names
report = classification_report(preds,predictions.label_ids,target_names = label_names)
print(report)
print("F1 Score:", f1_score(predictions.label_ids, preds,average = "weighted"))

0.9255
              precision    recall  f1-score   support

     sadness       0.97      0.95      0.96       591
         joy       0.95      0.94      0.95       697
        love       0.79      0.82      0.81       153
       anger       0.88      0.95      0.92       256
        fear       0.94      0.87      0.91       242
    surprise       0.74      0.80      0.77        61

    accuracy                           0.93      2000
   macro avg       0.88      0.89      0.88      2000
weighted avg       0.93      0.93      0.93      2000

F1 Score: 0.9250696112275184


In [50]:
##convertinglabelids to class names
# Map predicted label IDs to class names
pred_class_names = [label_names[i] for i in preds]
print(pred_class_names[:10])


['sadness', 'sadness', 'sadness', 'joy', 'sadness', 'fear', 'anger', 'joy', 'joy', 'anger']
